In [ ]:
import os
import base64
import langchain
# from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
# from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
# import wandb
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "chatrag"
os.environ["WANDB_NOTEBOOK_NAME"] = "retriever_agent_react_poc"

In [ ]:
with open("../data/secrets.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    os.environ["OPENAI_API_KEY"] = open_api_key
    if "organization_id" in secrets_dict.keys():
        openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
        os.environ["OPENAI_ORGANIZATION"] = openai_organization
        del open_api_key, openai_organization

In [ ]:
from chatrag.retriever import create_retriever_from_csv, create_retrieval_chain

## Data preparation

In [ ]:
retriever_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", max_tokens=2000)
# retriever_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"vote_average": "float"},
    llm=retriever_llm
)

In [ ]:
# retriever.get_relevant_documents("I want to watch a movie about space battles.")

# Vectorstore DB

In [ ]:
# retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", max_tokens=2000)

In [ ]:
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

In [ ]:
# print(g_media_sq_retrieval_chain.run("I want to watch a movie about space battles."))

# Tooling

In [ ]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence."""

In [ ]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    )
]

# Prompt templates

In [ ]:
# flake8: noqa
PREFIX = """You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.

Take a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS."""

In [ ]:
# chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_kwargs = {
        "system_message": PREFIX,
    }

# Chains

In [ ]:
# agent_llm_model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-16k", max_tokens=2000)
agent_llm_model = ChatOpenAI(temperature=0.7, model="gpt-4", max_tokens=2000)

In [ ]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    # agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True
)

# TESTS

### Run

In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I would like to watch a movie""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="I've changed my mind. Would you suggest me a good genre for just relaxing?.")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="Sounds good, give me a movie about it.")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="By the way, what's your name?.")
print("\nA partir de aquí la respuesta:\n")
print(response)